# Supervised Learning: Text preprocessing

In [216]:
### load packages 
import numpy as np 
import pandas as pd
from autocorrect import spell 
import matplotlib
from nltk.stem import WordNetLemmatizer 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import re
import string
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stopwords.words('english')
import warnings
from nltk import word_tokenize, corpus
warnings.simplefilter("ignore")
import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
sns.set() # use seaborn plotting style
from nltk.tokenize import word_tokenize

In [210]:
### load data and merge them  
title=pd.read_csv("title_sentiment.csv")
body=pd.read_csv("body_sentiment.csv")

## make clean text
title.rename({'clean_title': 'clean_text'}, axis=1, inplace=True)
body.rename({'clean_body': 'clean_text'}, axis=1, inplace=True)

#make new df with all text and sentiments 
df=pd.concat([title, body], axis=0)
# change column to string 
df['clean_text']=df['clean_text'].apply(str)

#look at ddta
print(df.info()) 
sents_count=pd.DataFrame(df.groupby("Sentiment")["Sentiment"].count()).rename(columns={"Sentiment":"Entries"}).reset_index().style.set_caption("Sentiment overview")
sents_count

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57885 entries, 0 to 19243
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentiment   57885 non-null  object
 1   score       57885 non-null  int64 
 2   comms_num   57885 non-null  int64 
 3   timestamp   57885 non-null  object
 4   clean_text  57885 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.6+ MB
None


,Sentiment,Entries
0,Negative,15690
1,Neutral,21017
2,Positive,21178


In [211]:
# dropping the columns we dont need
df.drop(columns=['score', 'comms_num', 'timestamp'], inplace=True)
df
print (list(df))
df['clean_text']=df['clean_text'].apply(str)


['Sentiment', 'clean_text']


In [212]:
def make_tokens(row):
    posts = row['clean_text']
    tokens = nltk.word_tokenize(posts)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

df['words'] = df.apply(make_tokens, axis=1)

#dropping the rows where the string is empty 
df=df[df['words'].map(lambda d: len(d)) > 0]


In [217]:
### Lemmatize

lemmatizer = WordNetLemmatizer() 

def Lemmatizer(row):
    my_list = row['words']
    lemma_list = [lemmatizer.lemmatize(word) for word in my_list]
    return (lemma_list)

df['lemmatized_words'] = df.apply(Lemmatizer, axis=1)



In [219]:
### removing stopwords 

stops = set(stopwords.words("english"))                  

def remove_stops(row):
    my_list = row['lemmatized_words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

df['lemma_meaningful'] = df.apply(remove_stops, axis=1)


In [220]:
### make back into string 

def make_string(row):
    my_list = row['lemma_meaningful']
    joined_words = ( " ".join(my_list))
    return joined_words

df['processed'] = df.apply(make_string, axis=1)

In [223]:
#drop irrelavant columns 
df.drop(columns=['clean_text', 'words', 'lemmatized_words'], inplace=True)


In [225]:
## Create csv 

# create csv file 
df.to_csv('processed_data.csv',index=False )